In [40]:
#두 문장간 유사도 측정(TF-IDF)
from pandas import DataFrame #Series, 

data = {}
data['gd']=[
  'MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI',
  'DVF DRESSVINCE JACKET',
  '(MP3)JBL PULSE 2 PORTABLE SPLASHPROOF BLUETOOTH SPEAKER ONE SIZE',
  'ARTICLES OF DAILY USE A GLASS OF WINE',
  'MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI AAA',  
  'CHAMPSSPORTS CHAMPSSPORTS NIKE AIR FLEECE PANTS'
]
data['weight']=[[0.1],[0.2],[0.3],[0.4],[0.5],[0.6]]
data['hs']=['0001', '0002', '0003', '0004', '0005', '0006']
#data['index']=[0,1,2,3,4]

dataFrame = DataFrame(data)
print(dataFrame)


                                                  gd weight    hs
0          MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI  [0.1]  0001
1                              DVF DRESSVINCE JACKET  [0.2]  0002
2  (MP3)JBL PULSE 2 PORTABLE SPLASHPROOF BLUETOOT...  [0.3]  0003
3              ARTICLES OF DAILY USE A GLASS OF WINE  [0.4]  0004
4      MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI AAA  [0.5]  0005
5    CHAMPSSPORTS CHAMPSSPORTS NIKE AIR FLEECE PANTS  [0.6]  0006


In [60]:
def cal_tfidf(dataFrame, new_text):
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(dataFrame['gd'])
    # overview에 대해서 tf-idf 수행
    print(tfidf_matrix.shape)

    add_tfidf_matirx = tfidf.transform([new_text])

    from sklearn.metrics.pairwise import linear_kernel
    cosine_sim = linear_kernel(add_tfidf_matirx, tfidf_matrix)
    print('cosine_sim:', cosine_sim)

    #중복제거
    import pandas as pd
    indices = pd.Series(dataFrame.index, index=dataFrame['gd']).drop_duplicates()
    
    return indices, cosine_sim

new_text = 'MEDICAL'
indices, cosine_sim = cal_tfidf(dataFrame, new_text)

(6, 28)
cosine_sim: [[0.40824829 0.         0.         0.         0.36546139 0.        ]]


In [0]:
def get_recommendations(dataFrame, title, indices, cosine_sim=cosine_sim):
    #loc = indices.get(title)
    #print('loc:', loc)

    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    #idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    #sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = list(enumerate(cosine_sim[0, :])) #첫번 row가 결과임

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 5개의 문장을 받아옵니다.
    sim_scores = sim_scores[0:0+5]  #0번째부터 받아옴
    print('sim_scores:', sim_scores)

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return dataFrame['gd'].iloc[movie_indices], sim_scores

In [78]:
#새로운 단어/문장 추가
new_text = 'MEDICAL'
indices, cosine_sim  = cal_tfidf(dataFrame, new_text)

top_gd, top_score  = get_recommendations(addDataFrame, new_text, indices, cosine_sim)

top_score = sorted(top_score, key=lambda x: x[1], reverse=True)
for idx, val in enumerate(top_score):
    print('num', (idx+1), ', gd:', top_gd[top_score[idx][0]], 'percentage:', top_score[idx][1])



(6, 28)
cosine_sim: [[0.40824829 0.         0.         0.         0.36546139 0.        ]]
sim_scores: [(0, 0.40824829046386296), (4, 0.36546138682094764), (1, 0.0), (2, 0.0), (3, 0.0)]
num 1 , gd: MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI percentage: 0.40824829046386296
num 2 , gd: MEDICAL EYE WASH SOLUTION 500ML KOBAYASHI AAA percentage: 0.36546138682094764
num 3 , gd: DVF DRESSVINCE JACKET percentage: 0.0
num 4 , gd: (MP3)JBL PULSE 2 PORTABLE SPLASHPROOF BLUETOOTH SPEAKER ONE SIZE percentage: 0.0
num 5 , gd: ARTICLES OF DAILY USE A GLASS OF WINE percentage: 0.0
